## Data Cleaning and Preprocessing Notebook

This notebook is to be strictly used for data cleaning and preprocessing purposes. Steps:

1. Read the dataset
2. Handle Missing Values (if any).
3. Do visualizations as required
4. Explore your data here
5. Save the cleaned and processed dataset as `data/final_dataset.csv`.
6. Split the dataset obtained in step 5 as `input/train.csv`,`input/test.csv`,`input/validation.csv`

NO MODELLING WILL BE DONE IN THIS NOTEBOOK!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import re
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit


In [4]:
df=pd.read_csv('../data/TARP_Project_Final_Dataset.csv')

In [5]:
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
df.shape

(61144, 3)

In [7]:
df['label'].value_counts()/len(df['label'])

REAL    0.523224
FAKE    0.476776
Name: label, dtype: float64

Before looking and wrangling at data, let's take a glance at the datasets

In [8]:
print("Tagged REAL:",df[df['label']=="REAL"]['title'].values[0])
print("Tagged FAKE:",df[df['label']=="FAKE"]['title'].values[0])

Tagged REAL: Kerry to go to Paris in gesture of sympathy
Tagged FAKE: You Can Smell Hillary’s Fear


In [9]:
df

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...
61139,Powell pushes diplomacy for N. Korea,WASHINGTON -- Outgoing Secretary of State Coli...,REAL
61140,Void is filled with Clement,With the supply of attractive pitching options...,REAL
61141,Martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...,REAL
61142,5 of arthritis patients in Singapore take Bext...,SINGAPORE : Doctors in the United States have ...,REAL


In [10]:
stops=set(stopwords.words('english'))

In [11]:
stops

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [12]:
def clean_text(text):
    """
    Cleans and preprocesses strings for NLP based tasks

    Args:
        text (str): Text that needs to be cleaned
    
    Returns:
        cleaned_text(str): The given text after cleaning and preprocessing
    """
    cleaned_text=text.strip() # Remove trailing whitespaces
    cleaned_text=text.lower() # Convert all to lowercase
    cleaned_text=word_tokenize(cleaned_text)
    cleaned_text=[word for word in cleaned_text if word not in stops]
    cleaned_text=" ".join(cleaned_text)
    cleaned_text=cleaned_text.replace("\n"," ") # Replace new lines with a space
    cleaned_text=re.sub(r'http[s]*://[A-Za-z0-9:./?=]*','url',cleaned_text) # Clean urls
    cleaned_text=re.sub(r"[^a-z\s]",'',cleaned_text) # Cleans out anything that is not a lowercase alphabet or a space
    cleaned_text=re.sub(r'[\s]+'," ",cleaned_text)
    return cleaned_text

In [13]:
" hello    hi".strip()

'hello    hi'

In [14]:
re.findall(r'http[s]*://[A-Za-z0-9:./?=]*',"https://t.co/VyTT49YvoE pic.twitter.com/wCvSCg4a5I But I am. this is my phone")

['https://t.co/VyTT49YvoE']

In [15]:
df.sample(5,random_state=10)['title'].apply(lambda x : clean_text(x))

37412    watch sean spicer fails spectacularly asked tr...
54257                         hewitt cruises quarterfinals
30131    tillerson warns region using lebanon proxy con...
38805    trump list unreported terror attacks absurd ci...
238      israel stole classified us information used he...
Name: title, dtype: object

In [16]:
df['text'].values[0]

'Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership is being warned that the entir

In [17]:
clean_text(df['text'].values[0])

'daniel greenfield shillman journalism fellow freedom center new york writer focusing radical islam final stretch election hillary rodham clinton gone war fbi word unprecedented thrown around often election ought retired still unprecedented nominee major political party go war fbi exactly hillary people done coma patients waking watching hour cnn hospital beds would assume fbi director james comey hillary opponent election fbi attack everyone obama cnn hillary people circulated letter attacking comey currently media hit pieces lambasting targeting trump surprising clintons allies start running attack ads fbi fbi leadership warned entire leftwing establishment form lynch mob continue going hillary fbi credibility attacked media democrats preemptively head results investigation clinton foundation hillary clinton covert struggle fbi agents obama doj people gone explosively public new york times compared comey j edgar hoover bizarre headline james comey role recalls hoover fbi fairly pract

In [18]:
df['title']=df['title'].apply(lambda x: clean_text(x))
df['text']=df['text'].apply(lambda x: clean_text(x))

In [20]:
df

,title,text,label,cleaned_title,cleaned_text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,smell hillary fear,daniel greenfield shillman journalism fellow f...
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,watch exact moment paul ryan committed politic...,google pinterest digg linkedin reddit stumbleu...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,kerry go paris gesture sympathy,us secretary state john f kerry said monday st...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,bernie supporters twitter erupt anger dnc we t...,kaydee king kaydeeking november lesson tonigh...
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,battle new york primary matters,s primary day new york frontrunners hillary cl...
...,...,...,...,...,...
61139,Powell pushes diplomacy for N. Korea,WASHINGTON -- Outgoing Secretary of State Coli...,REAL,powell pushes diplomacy n korea,washington outgoing secretary state colin l po...
61140,Void is filled with Clement,With the supply of attractive pitching options...,REAL,void filled clement,supply attractive pitching options dwindling d...
61141,Martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...,REAL,martinez leaves bitter,like roger clemens almost exactly eight years ...
61142,5 of arthritis patients in Singapore take Bext...,SINGAPORE : Doctors in the United States have ...,REAL,arthritis patients singapore take bextra cele...,singapore doctors united states warned painkil...


In [21]:
X=['cleaned_title','cleaned_text']
y='label'

In [27]:
def stratified_shuffle_split(df,X, y,test_size=0.3):
    """
    Generates a train and a test dataset using stratified sampling(the proportion of labels remains the same)
    from the parent dataset.

    Args:
        df (pandas.DataFrame): The pandas DataFrame object containing your data
        X (list/str): A string or list containing column(s) being used as features
        y (list/str): A string or list of target column(s)
        test_size (float, optional): Number between 0.0 and 1.0 indicating the proportion of the test data. Defaults to 0.3.

    Returns:
        (df_train,df_test): A tuple containing the given DataFrame split into two startified DataFrames
    """
    stratify=StratifiedShuffleSplit(test_size=test_size,random_state=42)
    for train_index, test_index in stratify.split(df[X],df[y]):
        df_train=df[X].iloc[train_index]
        df_test=df[y].iloc[test_index]
    return (df_train,df_test)

In [28]:
train,test=stratified_shuffle_split(df,X,y, test_size=0.4)
test,validation=stratified_shuffle_split(test,X,y,test_size=0.5)

KeyError: "None of [Index(['cleaned_title', 'cleaned_text'], dtype='object')] are in the [index]"

In [24]:
print(train.shape)
print(test.shape)
print(validation.shape)

(36686, 5)
(12229, 5)
(12229, 5)


In [25]:
train.to_csv('../input/train.csv')
test.to_csv('../input/test.csv')
validation.to_csv("../input/validation.csv")

In [26]:
train

,title,text,label,cleaned_title,cleaned_text
12075,Fox News’ Bill O’Reilly Calls for All Gun Crim...,"Email \nOn Wednesday evening Bill O’Reilly, ho...",FAKE,fox news bill reilly calls gun crimes federal ...,email wednesday evening bill reilly host reill...
56647,Junior swears by win at Talladega,Dale Earnhardt Jr. went from 11th on a restart...,REAL,junior swears win talladega,dale earnhardt jr went th restart lap first le...
49363,YOU CAN ADD INSIDER TRADING To The List Of Hil...,Wow! Yet more insider deals with the Clinton G...,FAKE,add insider trading list hillary crimes video,wow yet insider deals clinton grifters people ...
40681,Hannity Gets His A** Handed To Him For Falsel...,Fox News host Sean Hannity desperately tried t...,FAKE,hannity gets handed falsely claiming terence c...,fox news host sean hannity desperately tried s...
60071,Rwandan #39;motivated by riches #39;,Ouagadougou - Rwanda #39;s threat to launch mi...,REAL,rwandan motivated riches,ouagadougou rwanda threat launch military stri...
...,...,...,...,...,...
60492,Dutchman suspected of aiding Saddam in war cri...,A MAN suspected of helping former Iraqi leader...,REAL,dutchman suspected aiding saddam war crimes ar...,man suspected helping former iraqi leader sadd...
44289,Cop Holds Gun On Black Delivery Man During Ro...,Twenty-five-year-old Christopher Jeffries was ...,FAKE,cop holds gun black delivery man routine traff...,twentyfiveyearold christopher jeffries working...
29652,Suicide bombers kill 10 in Nigeria's Maiduguri...,"MAIDUGURI, Nigeria (Reuters) - Suicide bomb at...",REAL,suicide bombers kill nigeria s maiduguri emerg...,maiduguri nigeria reuters suicide bomb attacke...
12719,Conway: Called Delusional on “The View”,Conway: Called Delusional on “The View” Novemb...,FAKE,conway called delusional view,conway called delusional view november campaig...
